In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", 
                              temperature=0.1,
                              disable_streaming=False,
                              callbacks=[
                                  StreamingStdOutCallbackHandler(),
                              ])

In [3]:
t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country="France")


'What is the capital of France'

In [4]:
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)
t.format(country="France")


'What is the capital of France'

In [5]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [6]:
chat.predict("What do you know about France?")


C:\Users\rich\AppData\Local\Temp\ipykernel_37756\2023532840.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict("What do you know about France?")


"France is a country located in Western Europe.  Here's a summary of what I know about it, covering various aspects:\n\n**Geography and Demographics:**\n\n* **Location:**  It borders the Bay of Biscay, English Channel, Mediterranean Sea, and Atlantic Ocean.  It shares land borders with Belgium, Luxembourg, Germany, Switzerland, Italy, Monaco, Andorra, and Spain.\n* **Size:** It's a relatively large country, the largest in Western Europe.\n* **Landscape:**  France boasts diverse geography, including mountains (Alps, Pyrenees), plains, plateaus, and extensive coastlines.\n* **Population:**  It has a large and diverse population, with significant urban centers like Paris, Lyon, and Marseille.\n* **Climate:**  The climate varies regionally, from oceanic in the west to Mediterranean in the south and continental in the east.\n\n**Culture and History:**\n\n* **History:**  A country with a rich and long history, including periods of Roman rule, the Middle Ages, the Renaissance, the French Revo

In [8]:

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")


'\n    Human: What do you know about France?\n    AI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Italy?\n    AI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Greece?\n    AI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\n\nHuman: What do you know about Germany?'

In [9]:
chain = prompt | chat

chain.invoke({"country": "Germany"})


AIMessage(content='AI:\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Sauerkraut and Bratwurst\n        Currency: Euro\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-89c9529f-8e50-45ae-84b8-75e773553903-0', usage_metadata={'input_tokens': 155, 'output_tokens': 33, 'total_tokens': 188, 'input_token_details': {'cache_read': 0}})

In [ ]:
formatted_prompt = prompt.format(country="Germany")
answer = chat.stream(formatted_prompt)

for chunk in answer:
    print(chunk.content, end="", flush=True)



AIAI:
        I know this:
        Capital: Berlin
        Language::
        I know this:
        Capital: Berlin
        Language: German
        Food: Sauerkraut and Bratwurst
        Currency: Euro
 German
        Food: Sauerkraut and Bratwurst
        Currency: Euro


In [13]:
formatted_prompt = prompt.format(country="Turkey")
answer = chat.stream(formatted_prompt)

for chunk in answer:
    print(chunk.content, end="", flush=True)

AIAI:
        I know this:
        Capital: Ankara
        Language::
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira



In [14]:
chain = prompt | chat
result = chain.invoke({"country": "Turkey"})
print(result.content)


AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira

